In [1]:
transformation='JW'

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit import Aer

In [3]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
## HAMILTONIAN start
Molecule = 'H2'
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'

# Molecule = 'LiH'
# geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.44))]
# basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation=transformation)
### HAMILTONIAN end

#####################################

print(QubitHamiltonian)


fci_energy = Hamilt.molecule.fci_energy
print(fci_energy)

(-0.09706626816762878+0j) [] +
(-0.045302615503799264+0j) [X0 X1 Y2 Y3] +
(0.045302615503799264+0j) [X0 Y1 Y2 X3] +
(0.045302615503799264+0j) [Y0 X1 X2 Y3] +
(-0.045302615503799264+0j) [Y0 Y1 X2 X3] +
(0.1714128264477689+0j) [Z0] +
(0.16868898170361207+0j) [Z0 Z1] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.1659278503377034+0j) [Z0 Z3] +
(0.17141282644776892+0j) [Z1] +
(0.1659278503377034+0j) [Z1 Z2] +
(0.12062523483390414+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2] +
(0.1744128761226159+0j) [Z2 Z3] +
(-0.22343153690813558+0j) [Z3]
-1.137283834488502


From Helgaker, T., P. Jorgensen,and J. Olsen (2014),Molecularelectronic-structure theory(John Wiley & Sons)
we known

$$H_{2}^{ground} = 0.9939| 1100\rangle - 0.1106| 0011\rangle$$

From group theory we know only double excitation terms are important!
We can see this from our ijab operators that qubits 1 and 3 are only acted on by $Z$ therefore experience no population changes... Can even see this from the answer if we write it in BK basis:

$$H_{2}^{BK} = 0.9939| 1000\rangle - 0.1106| 0010\rangle$$

See how qubits 1 and 3 always remain at 0!

therefore can **remove** these measurements from Hamiltonian... as we know what expectation values they should give!

In the case for this Hamiltonian only $I$ and $Z$ act on these qubits! So even easier to remove (expectation values = +1 (as act on $| 0\rangle$ state!), so just remove and add to relivent H terms!

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [4]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=Hamilt.molecule.n_electrons
n_qubits=Hamilt.molecule.n_qubits

ansatz_obj = Ansatz(n_electrons,n_qubits)
print('JW ground state = ', ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print('BK ground state = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

JW ground state =  [1, 1, 0, 0]
BK ground state =  [1. 0. 0. 0.]


In [5]:
qubits_to_remove = Find_I_Z_indices_in_Hamiltonian(QubitHamiltonian, Hamilt.molecule.n_qubits)
print('qubits only acted on by I or Z:', qubits_to_remove)


input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# Remove_Z_terms_from_Hamiltonian
NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
                                                    QubitHamiltonian, 
                                                    input_state,
                                                    qubits_to_remove,
                                                    check_reduction=True)
NewQubitHamiltonian

qubits only acted on by I or Z: []


(-0.09706626816762878+0j) [] +
(-0.045302615503799264+0j) [X0 X1 Y2 Y3] +
(0.045302615503799264+0j) [X0 Y1 Y2 X3] +
(0.045302615503799264+0j) [Y0 X1 X2 Y3] +
(-0.045302615503799264+0j) [Y0 Y1 X2 X3] +
(0.1714128264477689+0j) [Z0] +
(0.16868898170361207+0j) [Z0 Z1] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.1659278503377034+0j) [Z0 Z3] +
(0.17141282644776892+0j) [Z1] +
(0.1659278503377034+0j) [Z1 Z2] +
(0.12062523483390414+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2] +
(0.1744128761226159+0j) [Z2 Z3] +
(-0.22343153690813558+0j) [Z3]

In [6]:
qubitNo_re_label_dict, NewQubitHamiltonian_relabelled = Re_label_Hamiltonian(NewQubitHamiltonian)
NewQubitHamiltonian_relabelled

(-0.09706626816762878+0j) [] +
(-0.045302615503799264+0j) [X0 X1 Y2 Y3] +
(0.045302615503799264+0j) [X0 Y1 Y2 X3] +
(0.045302615503799264+0j) [Y0 X1 X2 Y3] +
(-0.045302615503799264+0j) [Y0 Y1 X2 X3] +
(0.1714128264477689+0j) [Z0] +
(0.16868898170361207+0j) [Z0 Z1] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.1659278503377034+0j) [Z0 Z3] +
(0.17141282644776892+0j) [Z1] +
(0.1659278503377034+0j) [Z1 Z2] +
(0.12062523483390414+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2] +
(0.1744128761226159+0j) [Z2 Z3] +
(-0.22343153690813558+0j) [Z3]

# Find what new FCI energy is
- here should be the same as true answer!

In [7]:
from openfermion import qubit_operator_sparse
from scipy.sparse import csc_matrix
new_Molecular_H_MATRIX =  csc_matrix(qubit_operator_sparse(NewQubitHamiltonian_relabelled))

from scipy.sparse.linalg import eigs
try:
    eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
except:
    from scipy.linalg import eig
    eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
    
new_FCI_Energy = min(eig_values)


index = np.where(eig_values==new_FCI_Energy)[0][0]
ground_state_vector = eig_vectors[:, index]

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)
print(np.isclose(new_FCI_Energy, fci_energy))

new_FCI =  (-1.137283834488503+3.7121881080805557e-17j) VS old FCI: -1.137283834488502
True


# HF + UCCSD

In [8]:
input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
new_input_state = np.delete(input_state, qubits_to_remove)

n_qubits_new = len(new_input_state)
new_input_state

array([1, 1, 0, 0])

In [9]:
ansatz_obj.Get_ia_and_ijab_terms()

print('ia standard')
print(ansatz_obj.Sec_Quant_CC_ia_Fermi_ops)

print('ijab standard')
print(ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

### single trotter step
ansatz_obj.UCCSD_single_trotter_step(transformation,
                                     List_FermiOps_ia=ansatz_obj.Sec_Quant_CC_ia_Fermi_ops,
                                     List_FermiOps_ijab=ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

print('')
print('ia standard')
print(ansatz_obj.Second_Quant_CC_single_Trot_list_ia)
print('ijab standard')
print(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab)

ia standard
[-1.0 [0^ 2] +
1.0 [2^ 0], -1.0 [1^ 3] +
1.0 [3^ 1]]
ijab standard
[-1.0 [0^ 1^ 2 3] +
1.0 [3^ 2^ 1 0]]

ia standard
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]
ijab standard
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]


In [10]:
new_CC_ia_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ia,
                                                      qubits_to_remove)
print('ia reduced')
print(new_CC_ia_single_trot)

new_CC_ijab_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab,
                                                      qubits_to_remove)
print('ijab reduced')
print(new_CC_ijab_single_trot)


relabelled_new_CC_ia_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ia_single_trot)
relabelled_new_CC_ijab_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ijab_single_trot)

print('')
print('ia reduced relabelled')
print(relabelled_new_CC_ia_single_trot)
print('')
print('ijab reduced relabelled')
print(relabelled_new_CC_ijab_single_trot)

ia reduced
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]
ijab reduced
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]

ia reduced relabelled
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]

ijab reduced relabelled
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]


# Ansatz Circuit

In [11]:
# for op in relabelled_new_CC_ia_single_trot[1]:
#     print(op)
# print(list(op.terms.keys())[0])

In [12]:
# UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(new_input_state,
#                                      relabelled_new_CC_ia_single_trot, 
#                                      relabelled_new_CC_ijab_single_trot)

# theta_ia = [0 for _ in range(len(relabelled_new_CC_ia_single_trot))]
# theta_ijab = [0 for _ in range(len(relabelled_new_CC_ijab_single_trot))]

# UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                         Theta_param_list_ia=theta_ia, 
#                                          Theta_param_list_ijab=theta_ijab,
#                                          ia_first=True)
# UCCSD_ansatz_Q_Circ

# Graph

In [13]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(NewQubitHamiltonian_relabelled)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)

anti_commuting_sets

Building Graph Edges: 100%|##########| 15/15 [00:00<00:00, 1811.01it/s]


{0: [(0.12062523483390414+0j) [Z1 Z3]],
 1: [(0.1744128761226159+0j) [Z2 Z3]],
 2: [(0.16868898170361207+0j) [Z0 Z1]],
 3: [(-0.09706626816762878+0j) []],
 4: [(0.1659278503377034+0j) [Z0 Z3]],
 5: [(0.12062523483390414+0j) [Z0 Z2]],
 6: [(0.1659278503377034+0j) [Z1 Z2]],
 7: [(-0.22343153690813558+0j) [Z3], (0.045302615503799264+0j) [Y0 X1 X2 Y3]],
 8: [(-0.22343153690813564+0j) [Z2], (-0.045302615503799264+0j) [Y0 Y1 X2 X3]],
 9: [(0.1714128264477689+0j) [Z0], (-0.045302615503799264+0j) [X0 X1 Y2 Y3]],
 10: [(0.17141282644776892+0j) [Z1], (0.045302615503799264+0j) [X0 Y1 Y2 X3]]}

In [14]:
from qiskit.extensions import UnitaryGate
def Custom_gate_from_matrix(unitary_matrix, gate_label):
    
    return UnitaryGate(unitary_matrix, label='{}'.format(gate_label))
                       

# convert everything to qiskit circuits!
- note indexing goes from right to left in qiskit!

In [15]:
from quchem_ibm.Qiskit_Chemistry import *

In [16]:
q_reg = QuantumRegister(len(new_input_state))
qcirc = QuantumCircuit(q_reg)
HF_state_IBM_circuit(new_input_state, q_reg, qcirc).draw()

q0_0: ─────
           
q0_1: ─────
      ┌───┐
q0_2: ┤ X ├
      ├───┤
q0_3: ┤ X ├
      └───┘

In [558]:
def calc_exp_pauliword(count_dict, PauliWord):
    # takes correct part of bitstring when all lines measured

    qubitNos, PauliStrs = zip(*list(*PauliWord.terms.keys()))
    n_zeros = 0
    n_ones = 0

    for bitstring in count_dict:
        measure_term = np.take([int(bit) for bit in bitstring], qubitNos)

        parity_m_term = sum(measure_term) % 2

        if parity_m_term == 0:
            n_zeros += count_dict[bitstring]
        elif parity_m_term == 1:
            n_ones += count_dict[bitstring]
        else:
            raise ValueError('state {} not allowed'.format(measure_term))

    expectation_value = (n_zeros - n_ones) / (n_zeros + n_ones)

    return expectation_value

In [559]:
def calc_exp_pauliword_LCU(count_dict, PauliWord):
    # takes correct part of bitstring when all lines measured

    qubitNos, PauliStrs = zip(*list(*PauliWord.terms.keys()))
    
    
    # re-order IN REVERSE!!!!!!!!!!!!!! IMPORTANT!!!!!
    relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(n_qubits)[::-1])}
    qubitNos = [relabel_dic[qNo] for qNo in qubitNos]
    idx = np.argsort(qubitNos)
    qubitNos = np.array(qubitNos)[idx]
    PauliStrs = np.array(PauliStrs)[idx]
    ## !!!!!!!!!!!!! IMPORTANT!!!!!
    
    
    n_zeros = 0
    n_ones = 0

    for bitstring in count_dict:
        measure_term = np.take([int(bit) for bit in bitstring], qubitNos)

        parity_m_term = sum(measure_term) % 2

        if parity_m_term == 0:
            n_zeros += count_dict[bitstring]
        elif parity_m_term == 1:
            n_ones += count_dict[bitstring]
        else:
            raise ValueError('state {} not allowed'.format(measure_term))

    expectation_value = (n_zeros - n_ones) / (n_zeros + n_ones)

    return expectation_value

In [18]:
from tqdm.notebook import tqdm

In [19]:
len(ground_state_vector)

16

In [550]:
n_qubits = int(np.log2(len(ground_state_vector)))
backend=Aer.get_backend('qasm_simulator')
# n_shots=1024
n_shots=10_000

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)

perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector, q_reg, qcirc)

E_list=[]
# for qubitOp in NewQubitHamiltonian_relabelled:
for qubitOp in tqdm(NewQubitHamiltonian_relabelled, ascii=True, desc='performing VQE'):
    for PauliWord, const in qubitOp.terms.items():
        if PauliWord:
            full_circuit = change_basis_for_Z_measure(qubitOp, q_reg,
                                                      perfect_ansatz_circ,
                                                      n_qubits)
            full_circuit.measure_all()
            
            job = execute(full_circuit, backend, shots=n_shots)
            result = job.result()
            exp_counts_dict = result.get_counts(full_circuit)
            
            exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
            E_list.append(exp_val*const)
        else:
            E_list.append(const)
sum(E_list)

(-1.1375023116003495+0j)

In [30]:
def sparse_all_close(a,b, rtol=1e-10, atol=1e-8):
    c=np.abs(np.abs(a-b)-rtol*np.abs(b))
    return c.max()<=atol

from scipy.sparse import csr_matrix
test = csr_matrix([[1,2], [1,1]])
test2 = csr_matrix([[2,2], [0,1]])
sparse_all_close(test, test, atol=3)

True

In [ ]:
np.dot(test, test2)

In [28]:
from quchem.Unitary_partitioning_Seq_Rot import *
from scipy.sparse.linalg import expm
# from scipy.sparse import kron as sparse_kron


def Build_reduction_circuit_seq_rot_IBM(anti_commuting_set, S_index, N_qubits, check_reduction=False):
    """
    Function to build R_S (make up of all R_SK terms)

    Args:
        anti_commuting_set(list): list of anti commuting QubitOperators
        S_index(int): index for Ps in anti_commuting_set list
        check_reduction (optional, bool): use linear algebra to check that 𝑅s† 𝐻s 𝑅s == 𝑃s
    returns:
        full_RS_circuit(cirq.Circuit): Q_circuit for R_s operator
        Ps (QubitOperator): Pauli_S operator with cofactor of 1!
        gamma_l (float): normalization term

    """

    X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(anti_commuting_set, S_index)
    
    R_sl_mat_list=[]
    for X_sk_Op, theta_sk in X_sk_theta_sk_list:

        pauliword_X_sk = list(X_sk_Op.terms.keys())[0]
        const_X_sk = list(X_sk_Op.terms.values())[0]

        
        sparse_X_sk = get_sparse_operator(QubitOperator(pauliword_X_sk, -1j), n_qubits=N_qubits)
        R_sl_mat = expm(sparse_X_sk*(theta_sk / 2 * const_X_sk))
        R_sl_mat_list.append(R_sl_mat)

        
    R_S_matrix = reduce(np.dot, R_sl_mat_list[::-1]) # note need this in reverse order!

    if check_reduction:

        H_S=QubitOperator()
        for op in full_normalised_set['PauliWords']:
            H_S+=op
        H_S_matrix=get_sparse_operator(H_S, n_qubits=N_qubits)
        
        
        Ps_mat = get_sparse_operator(Ps, n_qubits=N_qubits)

        reduction_mat = R_S_matrix.dot(H_S_matrix.dot(R_S_matrix.conj().transpose()))
        
        if not (sparse_all_close(Ps_mat, reduction_mat)):
#         if not (np.allclose(Ps_mat.todense(), reduction_mat.todense())):
            print('reduction circuit incorrect...   𝑅s 𝐻s 𝑅s† != 𝑃s')

        
    return R_S_matrix, Ps, gamma_l


In [31]:
N_qubits = len(new_input_state)
S_index=0
set_key=9

R_s_mat, Ps, gamma_l=Build_reduction_circuit_seq_rot_IBM(anti_commuting_sets[set_key],
                                    S_index,
                                    N_qubits,
                                    check_reduction=True)

In [32]:
def IBM_gate_from_matrix(unitary_matrix, q_circuit, gate_str=None):
    
    if isinstance(unitary_matrix, scipy.sparse.csc.csc_matrix):
        unitary_matrix=unitary_matrix.toarray()
    
    
    q_circuit=q_circuit.copy()
    N_qubits = q_circuit.num_qubits
    
    if unitary_matrix.shape != (2**N_qubits, 2**N_qubits):
        raise ValueError('defined matrix is incorrect dimention')
    
    qubit_list = [i for i in range(N_qubits)]
    custom_gate = UnitaryGate(unitary_matrix, label='Custom Gate' if gate_str is None else '{}'.format(gate_str))
    
    del unitary_matrix
    q_circuit.append(custom_gate, qubit_list)
    return q_circuit

In [33]:
N_qubits = len(new_input_state)
q_reg = QuantumRegister(N_qubits)
qcirc = QuantumCircuit(q_reg)

qcirc = IBM_gate_from_matrix(R_s_mat, qcirc, gate_str='R_s')
qcirc.draw()

┌──────┐
q379_0: ┤0     ├
        │      │
q379_1: ┤1     ├
        │  R_s │
q379_2: ┤2     ├
        │      │
q379_3: ┤3     ├
        └──────┘

In [119]:
#Changing the simulator 
backend = Aer.get_backend('unitary_simulator')

#job execution and getting the result as an object
job = execute(qcirc, backend)
result = job.result()

#get the unitary matrix from the result object
unitary_form = result.get_unitary(qcirc, decimals=6)

np.allclose(unitary_form, np.around(R_s_mat.todense(), 6))

False

In [120]:
Seq_Rot_circuits=[]
N_qubits = len(new_input_state)
S_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector, q_reg, qcirc)

E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:
        R_s_mat, Ps, gamma_l=Build_reduction_circuit_seq_rot_IBM(anti_set_list,
                                                                S_index,
                                                                N_qubits,
                                                                check_reduction=False)
        # unitary partitioning
        q_circuit = IBM_gate_from_matrix(R_s_mat, perfect_ansatz_circ, gate_str='R_s')
        
        full_circuit = change_basis_for_Z_measure(Ps, 
                                                  q_reg,
                                                  q_circuit,
                                                  N_qubits)
        full_circuit.measure_all()

        job = execute(full_circuit, backend, shots=n_shots)
        result = job.result()
        exp_counts_dict = result.get_counts(full_circuit)
        E_list.append(exp_val*gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                full_circuit = change_basis_for_Z_measure(qubitOp, 
                                                          q_reg,
                                                          perfect_ansatz_circ,
                                                          N_qubits)
                full_circuit.measure_all()

                job = execute(full_circuit, backend, shots=n_shots)
                result = job.result()
                exp_counts_dict = result.get_counts(full_circuit)

                exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
                E_list.append(exp_val*const)
            else:
                E_list.append(const)
print(sum(E_list))


(-1.1376232075920538+0j)


# LCU

In [41]:
from quchem.Unitary_partitioning_LCU_method import *

In [282]:
R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_commuting_sets[9], 0)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)
R_corr_list

[1, (-0-1j)]

In [283]:
No_control_qubits=int(np.log2(len(ancilla_amplitudes)))
No_system_qubits=int(np.log2(len(ground_state_vector)))

R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_commuting_sets[9], 0)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)



R_circ_obj = LCU_R_gate(No_control_qubits, No_system_qubits, R_corrected_Op_list, R_corr_list, Pn)
R_circ_circ = cirq.Circuit(
cirq.decompose_once((R_circ_obj(*cirq.LineQubit.range(R_circ_obj.num_qubits())))))
R_circ_circ

0: ───(-0-1j)*Y0───
      │
1: ───X1───────────
      │
2: ───Y2───────────
      │
3: ───Y3───────────
      │
4: ───@────────────

In [310]:
# LCU_circuits=[]
N_qubits = len(new_input_state)
N_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector, q_reg, qcirc)





N_ancilla=int(np.log2(len(ancilla_amplitudes)))
q_reg_ancilla = QuantumRegister(N_ancilla)
q_circ_ancilla = QuantumCircuit(q_reg_ancilla)
G_circuit = Get_Q_circ_to_build_state(ancilla_amplitudes, q_reg_ancilla, q_circ_ancilla)
G_inverse=G_circuit.inverse()

combined_circuits =perfect_ansatz_circ.combine(G_circuit)
print(combined_circuits.draw())

R_l_LCU, N_ancil = control_Rl_gate_LCU(R_corrected_Op_list,
                                       R_corr_list,
                                       ancilla_amplitudes,
                                       int(np.log2(len(ground_state_vector))))


R_circ_obj = LCU_R_gate(No_control_qubits, No_system_qubits, R_corrected_Op_list, R_corr_list, Pn)
R_circ_circ = cirq.Circuit(
cirq.decompose_once((R_circ_obj(*cirq.LineQubit.range(R_circ_obj.num_qubits())))))
print(R_circ_circ)
q_circuit = IBM_gate_from_matrix(R_circ_circ.unitary(), combined_circuits, gate_str='R_l_LCU')
        
# full_circuit = change_basis_for_Z_measure(Ps, 
#                                           q_reg,
#                                           q_circuit,
#                                           N_qubits)
q_circuit.draw()

         ┌──────────────────┐
q2222_0: ┤0                 ├
         │                  │
q2222_1: ┤1                 ├
         │  disentangler_dg │
q2222_2: ┤2                 ├
         │                  │
q2222_3: ┤3                 ├
         ├─────────────────┬┘
q2276_0: ┤ disentangler_dg ├─
         └─────────────────┘ 
0: ───(-0-1j)*Y0───
      │
1: ───X1───────────
      │
2: ───Y2───────────
      │
3: ───Y3───────────
      │
4: ───@────────────


┌──────────────────┐┌──────────┐
q2222_0: ┤0                 ├┤0         ├
         │                  ││          │
q2222_1: ┤1                 ├┤1         ├
         │  disentangler_dg ││          │
q2222_2: ┤2                 ├┤2 R_l_LCU ├
         │                  ││          │
q2222_3: ┤3                 ├┤3         ├
         ├─────────────────┬┘│          │
q2276_0: ┤ disentangler_dg ├─┤4         ├
         └─────────────────┘ └──────────┘

In [311]:
from qiskit.quantum_info import Operator

q_reg = QuantumRegister(5)
qcirc = QuantumCircuit(5)
test = IBM_gate_from_matrix(R_circ_circ.unitary(), qcirc, gate_str='R_l_LCU')
Operator(test)==R_circ_circ.unitary()

False

In [467]:
def phase_Pauli_gate(Pstr, cofactor):
    if Pstr == 'X':
        unitary_matrix = cofactor * np.array([[0, 1], [1, 0]], dtype=complex)
    elif Pstr == 'Y':
        unitary_matrix = cofactor * np.array([[0, -1j], [1j, 0]], dtype=complex)
    elif Pstr == 'Z':
        unitary_matrix = cofactor * np.array([[1, 0], [0, -1]], dtype=complex)
    else:
        raise ValueError('P_str is not a Pauli')

    return UnitaryGate(unitary_matrix, label='{}*{}'.format(cofactor, Pstr))

In [ ]:
>>> a = [1,2,3,4,5]
>>> b = [1,3,5,6]
>>> list(set(a) & set(b))
[1, 3, 5]

In [534]:
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate
def control_P_IBM(PauliOp, phase_correction, control_index, q_circuit, n_qubits, n_ancilla, list_measured_qubits=None):
    
    q_circuit = q_circuit.copy()
    qubitNos, PauliStrs = zip(*list(*PauliOp.terms.keys()))
    
    # re-order IN REVERSE!!!!!!!!!!!!!! IMPORTANT!!!!!
    relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(n_qubits)[::-1])}
    qubitNos = [relabel_dic[qNo] for qNo in qubitNos]
    idx = np.argsort(qubitNos)
    qubitNos = np.array(qubitNos)[idx]
    PauliStrs = np.array(PauliStrs)[idx]
    ## !!!!!!!!!!!!! IMPORTANT!!!!!
    
    if list_measured_qubits is None:
        qubit_to_put_phase_on=0
    else:
        qubit_to_put_phase_on = list(set(qubitNos) & set(list_measured_qubits))[0]
        
    
    for index, qNo in enumerate(qubitNos):
#         qNo = int(qNo)
        qNo=qNo+n_ancilla
        Pstr = PauliStrs[index]
        
        if index==qubit_to_put_phase_on:
            phase_P = phase_Pauli_gate(Pstr, phase_correction).control(n_ancilla)
            phase_P.num_ctrl_qubits =n_ancilla
            phase_P.ctrl_state= control_index
            q_circuit.append(phase_P, [*[i for i in range(0, n_ancilla)],qNo])
#             Matrix_P = Operator(phase_P)
        else:
            if Pstr == 'X':
                X_gate = XGate().control(n_ancilla)
                X_gate.ctrl_state= control_index
    #             q_circuit.append(X_gate, [*[i for i in range(n_qubits, n_qubits+n_ancilla)],qNo])
                q_circuit.append(X_gate, [*[i for i in range(0, n_ancilla)],qNo])
            elif Pstr == 'Y':
                Y_gate = YGate().control(n_ancilla)
                Y_gate.ctrl_state= control_index
    #             q_circuit.append(Y_gate, [*[i for i in range(n_qubits, n_qubits+n_ancilla)],qNo])
                q_circuit.append(Y_gate, [*[i for i in range(0, n_ancilla)],qNo])
            elif Pstr == 'Z':
                Z_gate = ZGate().control(n_ancilla)
                Z_gate.ctrl_state= control_index
    #             q_circuit.append(Z_gate, [*[i for i in range(n_qubits, n_qubits+n_ancilla)],qNo])
                q_circuit.append(Z_gate, [*[i for i in range(0, n_ancilla)],qNo])
            
    return q_circuit

In [538]:
system=3
ancilla=2
q_reg = QuantumRegister(system+ancilla)
qcirc = QuantumCircuit(q_reg)
control_index=2
list_qubits_to_measure = [1]

P = QubitOperator('X0 Y1 Z2', 1)
phase = 1j

xx =control_P_IBM(P, phase, control_index, qcirc, system, ancilla, list_measured_qubits=list_qubits_to_measure)
xx.draw()

q5876_0: ─o────o──────o──
          │    │      │  
q5876_1: ─■────■──────■──
          │    │      │  
q5876_2: ─■────┼──────┼──
            ┌──┴───┐  │  
q5876_3: ───┤ 1j*Y ├──┼──
            └──────┘┌─┴─┐
q5876_4: ───────────┤ X ├
                    └───┘

In [490]:
# LCU_circuits=[]
N_qubits = len(new_input_state)
N_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector, q_reg, qcirc)





N_ancilla=int(np.log2(len(ancilla_amplitudes)))
q_reg_ancilla = QuantumRegister(N_ancilla)
q_circ_ancilla = QuantumCircuit(q_reg_ancilla)
G_circuit = Get_Q_circ_to_build_state(ancilla_amplitudes, q_reg_ancilla, q_circ_ancilla)
G_inverse=G_circuit.copy().inverse()

# combined_circuits =perfect_ansatz_circ.combine(G_circuit)
combined_circuits =G_circuit.combine(perfect_ansatz_circ)
print(combined_circuits.draw())

R_l_LCU, N_ancil = control_Rl_gate_LCU(R_corrected_Op_list,
                                       R_corr_list,
                                       ancilla_amplitudes,
                                       int(np.log2(len(ground_state_vector))))


control_index=1
q_circuit =control_P_IBM(R_corrected_Op_list[1], R_corr_list[1], control_index, combined_circuits, N_qubits, N_ancilla)

        
# full_circuit = change_basis_for_Z_measure(Ps, 
#                                           q_reg,
#                                           q_circuit,
#                                           N_qubits)
q_circuit.draw()

combined_circuits =q_circuit.combine(G_inverse)
full_circuit = change_basis_for_Z_measure(Ps, 
                                          q_reg,
                                          combined_circuits,
                                          N_qubits)
full_circuit.draw()

         ┌─────────────────┐ 
q3535_0: ┤ disentangler_dg ├─
         ├─────────────────┴┐
q3481_0: ┤0                 ├
         │                  │
q3481_1: ┤1                 ├
         │  disentangler_dg │
q3481_2: ┤2                 ├
         │                  │
q3481_3: ┤3                 ├
         └──────────────────┘


┌─────────────────┐                             ┌────────────────────┐
q3535_0: ┤ disentangler_dg ├───────■────────■────■────■──┤ disentangler_dg_dg ├
         ├─────────────────┴┐┌─────┴─────┐  │    │    │  └────────────────────┘
q3481_0: ┤0                 ├┤ (-0-1j)*X ├──┼────┼────┼────────────────────────
         │                  │└───────────┘┌─┴─┐  │    │                        
q3481_1: ┤1                 ├─────────────┤ Y ├──┼────┼────────────────────────
         │  disentangler_dg │             └───┘┌─┴─┐  │                        
q3481_2: ┤2                 ├──────────────────┤ X ├──┼────────────────────────
         │                  │                  └───┘┌─┴─┐                      
q3481_3: ┤3                 ├───────────────────────┤ X ├──────────────────────
         └──────────────────┘                       └───┘

In [491]:
qcirc = G_circuit.copy()
print(qcirc.draw())
# qcirc.measure_all()

#Changing the simulator 
backend = Aer.get_backend('statevector_simulator')

#job execution and getting the result as an object
job = execute(qcirc, backend)
result = job.result()

print('expected amps =', ancilla_amplitudes)
result.get_statevector(qcirc)

         ┌─────────────────┐
q3535_0: ┤ disentangler_dg ├
         └─────────────────┘
expected amps = [0.9407564775082788, 0.3390829544908076]


array([0.94075648+0.j, 0.33908295+0.j])

In [499]:
'01234567'[:-2]

'012345'

In [520]:
def Get_post_selection_counts_LCU(list_of_measurements, N_ancilla):
    # checks all zero state on ancilla line

    new_counts={}
    for binary_result_str in list_of_measurements:
        ancilla_state = int(binary_result_str[-N_ancilla:],2)
        if ancilla_state==0:
            post_select_m_binary = binary_result_str[:-N_ancilla]
            if post_select_m_binary in new_counts.keys():
                new_counts[post_select_m_binary]+=1
            else:
                new_counts[post_select_m_binary]=1
            
        else:
            continue

    return new_counts

In [522]:
Get_post_selection_counts_LCU(['0000', '0001', '1100', '0111'], 2)

{'00': 1, '11': 1}

In [561]:
# LCU_circuits=[]
N_qubits = len(new_input_state)
N_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(N_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=arb_state_initalize_circuit(ground_state_vector, q_reg, qcirc)
N_SYSTEM=int(np.log2(len(ground_state_vector)))

E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:

        R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_set_list, N_index)
        R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)
        # unitary partitioning
        
        N_ancilla=int(np.log2(len(ancilla_amplitudes)))
        q_reg_ancilla = QuantumRegister(N_ancilla)
        q_circ_ancilla = QuantumCircuit(q_reg_ancilla)
        G_circuit = Get_Q_circ_to_build_state(ancilla_amplitudes, q_reg_ancilla, q_circ_ancilla)
        G_inverse=G_circuit.copy().inverse()
        
        # combine ancilla and system
        combined_circuits =G_circuit.combine(perfect_ansatz_circ)
        R_l_LCU, N_ancilla = control_Rl_gate_LCU(R_corrected_Op_list,
                                       R_corr_list,
                                       ancilla_amplitudes,
                                       N_SYSTEM)
        
        
        # find qubits that are measured!
        Pn_qubitNos, _ = zip(*list(*Pn.terms.keys()))
        # re-order IN REVERSE!!!!!!!!!!!!!! IMPORTANT!!!!!
        relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(N_SYSTEM)[::-1])}
        Pn_qubitNos = [relabel_dic[qNo_Pn] for qNo_Pn in Pn_qubitNos]
        idx = np.argsort(Pn_qubitNos)
        Pn_qubitNos = np.array(Pn_qubitNos)[idx]
        
        
        for control_index, op in enumerate(R_corrected_Op_list):
            phase_corr=R_corr_list[control_index]
            for PauliW, Const in op.terms.items():
                if PauliW:
                    combined_circuits =control_P_IBM(op,
                                                     phase_corr,
                                                     control_index,
                                                     combined_circuits,
                                                     N_qubits, 
                                                     N_ancilla,
                                                    list_measured_qubits=Pn_qubitNos)
                else:
                    continue
        
        full_circuit = change_basis_for_Z_measure(Pn, 
                                                  q_reg,
                                                  combined_circuits,
                                                  N_qubits)
        full_circuit.measure_all()
        
        
        job = execute(full_circuit, backend, shots=n_shots, memory=True) # need memory for post_selection!
        result = job.result()
        list_of_results=result.get_memory(full_circuit)
        post_selected_dict = Get_post_selection_counts_LCU(list_of_results, N_ancilla)
        exp_val = calc_exp_pauliword_LCU(post_selected_dict, Pn)
        
        print(gamma_l*exp_val)
        print(full_circuit.draw())
        print(Pn)
        
        E_list.append(exp_val*gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                full_circuit = change_basis_for_Z_measure(qubitOp, 
                                                          q_reg,
                                                          perfect_ansatz_circ,
                                                          N_qubits)
                full_circuit.measure_all()

                job = execute(full_circuit, backend, shots=n_shots)
                result = job.result()
                exp_counts_dict = result.get_counts(full_circuit)

                exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
                E_list.append(exp_val*const)
            else:
                E_list.append(const)
print(sum(E_list))

(-0.2219786006707709+0j)
         »
q7024_0: »
         »
q7023_0: »
         »
q7023_1: »
         »
q7023_2: »
         »
q7023_3: »
         »
 meas_0: »
         »
 meas_1: »
         »
 meas_2: »
         »
 meas_3: »
         »
 meas_4: »
         »
«                                        ┌─────────────────┐                               »
«q7024_0: ───────────────────────────────┤ disentangler_dg ├───────────────────────────────»
«         ┌──────────────────────────────┴─────────────────┴──────────────────────────────┐»
«q7023_0: ┤0                                                                              ├»
«         │                                                                               │»
«q7023_1: ┤1                                                                              ├»
«         │  initialize(0,0,0,0.076769+0.082296j,0,0,0,0,0,0,0,0,-0.6778+-0.72659j,0,0,0) │»
«q7023_2: ┤2                                                                              ├»


(0.17300379942790103+0j)
         »
q7100_0: »
         »
q7023_0: »
         »
q7023_1: »
         »
q7023_2: »
         »
q7023_3: »
         »
 meas_0: »
         »
 meas_1: »
         »
 meas_2: »
         »
 meas_3: »
         »
 meas_4: »
         »
«                                        ┌─────────────────┐                               »
«q7100_0: ───────────────────────────────┤ disentangler_dg ├───────────────────────────────»
«         ┌──────────────────────────────┴─────────────────┴──────────────────────────────┐»
«q7023_0: ┤0                                                                              ├»
«         │                                                                               │»
«q7023_1: ┤1                                                                              ├»
«         │  initialize(0,0,0,0.076769+0.082296j,0,0,0,0,0,0,0,0,-0.6778+-0.72659j,0,0,0) │»
«q7023_2: ┤2                                                                              ├»


In [ ]:
def control_P_gate(PauliOp, cofactor, control_string_binary, n_qubits, n_ancilla):
    
    qubit_state = {'0': np.array([[1],[0]]), '1': np.array([[0],[1]])}
    
    ancilla_state = reduce(np.kron, [qubit_state[bit]for bit in control_string_binary])
    
    
    if Pstr == 'X':
        unitary_matrix = cofactor * np.array([[0, 1], [1, 0]], dtype=complex)
    elif Pstr == 'Y':
        unitary_matrix = cofactor * np.array([[0, -1j], [1j, 0]], dtype=complex)
    elif Pstr == 'Z':
        unitary_matrix = cofactor * np.array([[1, 0], [0, -1]], dtype=complex)
    else:
        raise ValueError('P_str is not a Pauli')

    return UnitaryGate(unitary_matrix, label='{}*{}'.format(cofactor, Pstr)).control(1)

In [ ]:
A = np.eye(16)
A[:,0].shape

In [ ]:
A = np.eye(16)
A[:,0]=ground_state_vector
A[0,1]

In [ ]:
def normalize(v):
    return v / np.sqrt(v.dot(v))

n = len(A)

# A[:, 0] = normalize(A[:, 0])

for i in range(1, n):
    Ai = A[:, i]
    for j in range(0, i):
        Aj = A[:, j]
        t = Ai.dot(Aj)
        Ai = Ai - t * Aj
#     A[:, i] = normalize(Ai)
    A[:, i] = Ai


In [ ]:
sum(np.abs(i)**2 for i in ground_state_vector)

In [ ]:
ground_state_vector

In [ ]:
np.trace(np.dot(A, A.conj().T))

In [ ]:
xx = normalize(ground_state_vector)

np.allclose(ground_state_vector, xx)

In [ ]:
def normalize(v):
    mag = sum(np.abs(v**2))
    return v / np.sqrt(mag)

def Get_Unitary_arb_state(arb_state):
    
    n_qubits = len(arb_state)
    
    A = np.eye(n_qubits, dtype=complex)
    A[:,0]=arb_state
    
    n = len(A)
    for i in range(1, n):
        Ai = A[:, i]
        for j in range(0, i):
            Aj = A[:, j]
            t = Ai.dot(Aj)
            Ai = Ai - t * Aj
        A[:, i] = normalize(Ai)
        
    return A
    

In [ ]:
AA = Get_Unitary_arb_state(ground_state_vector)


np.allclose(ground_state_vector, AA[:,0])

In [ ]:
A = np.eye(16, dtype=complex)
A[:,0]=ground_state_vector

q, r = np.linalg.qr(A, mode='complete')

In [ ]:
np.allclose(q.dot(q.conj().T), np.eye(16, dtype=complex), atol=1e-1)

In [ ]:
x = np.linalg.inv(r) * (q.T) * A
x

In [ ]:
def normalize(v):
    return v / np.sqrt(v.dot(v))

n = len(A)

A[:, 0] = normalize(A[:, 0])

for i in range(1, n):
    Ai = A[:, i]
    for j in range(0, i):
        Aj = A[:, j]
        t = Ai.dot(Aj)
        Ai = Ai - t * Aj
    A[:, i] = normalize(Ai)


In [ ]:
A[:, 1].dot(A[:, 2])

In [ ]:
np.sum(np.abs(unitary_check[:,4])**2)

In [ ]:
np.allclose(unitary_check, np.eye(16, dtype=complex), atol=1e-1)

In [ ]:
unitary_check[0,12]

In [ ]:
np.where((np.around(unitary_check, 1) == np.eye(16)) == False)

In [ ]:
x = np.array([1,0])
np.inner(x,x)

In [ ]:
def normalize(v):
    mag = sum(np.abs(v**2))
    return v / np.sqrt(mag)

def Get_Unitary_arb_state(arb_state):
    
    n_qubits = len(arb_state)
    
    A = np.eye(n_qubits, dtype=complex)
    
    A[:,0]=arb_state
    
    n = len(A)
    for i in range(1,n):
        ortho_vec = A[:,i] - reduce(sum, [ np.inner(A[:,j], A[:,i])* A[:,j] for j in range(i)])
        ortho_vec=normalize(ortho_vec)
        print(np.dot(ortho_vec, A[:,i]))
        
Get_Unitary_arb_state(ground_state_vector)

In [ ]:
from scipy.stats import unitary_group

X = unitary_group.rvs(3)

A = np.eye(3, dtype=complex)
A[:,0]=normalize(X[:,0])

# n = len(A)
# for i in range(1,n):
    
#     ortho_vec = A[:,i] - reduce(sum, [ np.inner(A[:,j], A[:,i])* A[:,j] for j in range(i)])
#     ortho_vec=normalize(ortho_vec)
#     print(np.dot(ortho_vec, A[:,i]))

In [ ]:
y2

In [ ]:
v1 = X[:,0]
v2 = X[:,1]

y2 = v2 - np.inner(v2, v1)*v1
y2 = normalize(y2)
v1.dot(y2)

In [ ]:
v1 = (1/np.sqrt(2)) * np.array([-1,1,0])
v2 = np.array([-1,0,1])

y2 = v2 - np.inner(v2, v1)*v1
y2 = normalize(y2)

v1.dot(y2)

In [ ]:
v1

In [ ]:
v1*v2_new

In [ ]:
np.dot(A[:,0], B.reshape([3,1]))

In [ ]:
C = np.hstack((np.eye(3), np.array([[4],[6],[5]])))
C

In [ ]:
np.ufunc.reduce(C)

In [ ]:
u1 = np.array([[4],[6],[5]])
u1

In [ ]:
v = np.array([[1],[0],[0]])

u2 = np.matmul(u1,v)
u2

In [ ]:
x = np.array([4, 6, 5])
y = np.array([1, 0, 0])
w=np.cross(x,y)

w.dot(x)

In [ ]:
np.outer(x,y)
w

In [ ]:
w

In [ ]:
z = np.cross(w,x)
z.dot(w)

In [ ]:
AA = np.hstack((x.reshape(3,1),y.reshape(3,1),z.reshape(3,1)))
AA

In [ ]:
def normalize(v):
    mag = sum(np.abs(v**2))
    return v / np.sqrt(mag)

def Get_arb_state_matrix(arb_state):
    
    arb_state=normalize(arb_state)
#     if sum(np.abs(arb_state**2))!=1:
#         raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    

    
    A= np.zeros((n_qubits,n_qubits), dtype=complex)
    A[:,0]=arb_state
    
    v=np.zeros(n_qubits)
    v[0]=1
    A[:,1]=np.cross(A[:,0],v)
    
    for i in range(2, n_qubits):
        A[:,i]=np.cross(A[:,i-1], A[:,i-2])
    
    return A


In [ ]:
W = Get_arb_state_matrix(np.array([4, 6, 5, 7]))

In [ ]:
x1 = np.array([[4], [6], [5]])
x1=normalize(x1)
x1

In [ ]:
x2=np.array([[0], [1], [0]])

In [ ]:
x2_new= x2 - np.dot(x1.T, x2)*x1

np.dot(x1.T, x2_new)

In [ ]:
x2_new

In [ ]:
x3=np.array([[0], [0], [1]])

In [ ]:
x3_new= x3 - (np.dot(x1.T, x3)*x1 + np.dot(x2_new.T, x3)*x2_new)

np.dot(x1.T, x3_new)

In [ ]:
np.dot(x2_new.T, x3_new)

In [ ]:
x2_new

In [ ]:
def Gran_Shmidt(arb_state):
    # https://stackoverflow.com/questions/12327479/how-to-build-a-ortoghonal-basis-from-a-vector
    
    arb_state=normalize(arb_state)
#     if sum(np.abs(arb_state**2))!=1:
#         raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    A= np.eye(n_qubits, dtype=complex)
    A[:,0]=arb_state
    
    for i in range(1, n_qubits):
        project = reduce(sum, [np.inner(A[:,i], A[:,j])*A[:,j] for j in range(i)])
        
        A[:,i]=normalize(A[:,i]-project)
        
        print(np.inner(A[:,i], A[:,i-1]))
        
    return A
    

In [ ]:
WW=Gran_Shmidt(np.array([1,1,0,1]))
print(WW)

np.around(WW.conj().T.dot(WW), 3)

In [ ]:
from scipy.linalg import null_space
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.null_space.html
coef
Z = null_space(B)

In [ ]:
# from scipy.linalg import svd

In [ ]:
A = np.eye(3)
A[:,0]= np.array([np.sqrt(1/2), 0,np.sqrt(1/2)])
A

In [ ]:
A = np.random.rand(3, 3)
project(A[:,0], A[:,1])

In [ ]:
for i in range(1, 3):
    projection = reduce(sum, [project(A[:,j], A[:,i])for j in range(i)])
        
    A[:,i]=normalize(A[:,i]-projection)

    

A


In [ ]:
def project(u,v):
    coef = v.conj().T.dot(u)/ u.conj().T.dot(u)
    return coef*u

def Gram_Schimdt(arb_state):
    # https://stackoverflow.com/questions/12327479/how-to-build-a-ortoghonal-basis-from-a-vector
    
#     arb_state=normalize(arb_state)
#     if sum(np.abs(arb_state**2))!=1:
#         raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    A= np.eye(n_qubits, dtype=complex)
    A[:,0]=arb_state
    
    for i in range(1, n_qubits):
        projection = reduce(sum, [project(A[:,i], A[:,j])for j in range(i)])
        A[:,i]=A[:,i]-projection
        
    return A

In [ ]:
BB = Gram_Schimdt(np.array([np.sqrt(1/2), 0, np.sqrt(1/2)]))

np.inner(BB[:,1], BB[:,2])
BB

In [ ]:
V = np.eye(4)
# V[:,0]= np.array([np.sqrt(1/3), np.sqrt(1/3),np.sqrt(1/3)])
V[:,0]= np.array([np.sqrt(1/2), 0,np.sqrt(1/2),-])

U=np.zeros(V.shape)
U[:,0]=V[:,0]

for i in range(1,A.shape[0]):
    U[:,i]=V[:,i]
    for j in range(i):
        U[:,i]=U[:,i]-(U[:,j].T.dot(U[:,i])/(U[:,j].T.dot(U[:,j])))*U[:,j]
        
U

In [ ]:
import random

In [ ]:
xx=ground_state_vector.real
mag = sum(np.abs(xx**2))
new=xx / np.sqrt(mag)

In [ ]:
from scipy.linalg import svd
def Gram_Schimdt(arb_state):
    
    # Get an orthonormal basis from a single vector (defines first column of output!)
    
    #WORKING
    # https://stackoverflow.com/questions/12327479/how-to-build-a-ortoghonal-basis-from-a-vector
    
    arb_state = np.ravel(arb_state)
    
    if not np.isclose(sum(np.abs(arb_state**2)),1):
        raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    V= np.eye(n_qubits, dtype=complex)
    V[:,0]=arb_state
    
    U=np.zeros(V.shape, dtype=complex)
    U[:,0]=V[:,0]
    for i in range(1,V.shape[0]):
        U[:,i]=V[:,i]
        for j in range(i):
            U[:,i]=U[:,i]-(U[:,j].T.dot(U[:,i])/(U[:,j].T.dot(U[:,j])))*U[:,j]
        
    Unitary_matrx, s, Vh= svd(U)
    
    # correct the sign
    if not np.allclose(Unitary_matrx[:,0], arb_state):
        Unitary_matrx[:,0]=Unitary_matrx[:,0]*-1
    
    if not np.allclose(Unitary_matrx[:,0], arb_state):
        raise ValueError('incorrect state generated')
    return Unitary_matrx


# arb_input_state=np.array([np.sqrt(1/3), np.sqrt(1/3), np.sqrt(1/3)], dtype=complex)
N_ancilla_qubits=4
arb_input_state=np.array([random.uniform(0, 1) for _ in range(2**N_ancilla_qubits)], dtype=complex)
arb_input_state = arb_input_state/np.sqrt(arb_input_state.dot(arb_input_state)) # NORMALIZE!

Unitary_to_make_state = Gram_Schimdt(arb_input_state)

print('unitary check:', np.allclose(Unitary_to_make_state.dot(Unitary_to_make_state.T), np.eye(len(arb_input_state))))
print('correct state check:',np.allclose(Unitary_to_make_state[:,0], arb_input_state))

In [ ]:
def Arb_state_U_gate(theta):
    Unitary_Matrix = np.array([
                    [np.cos(theta), np.sin(theta)],
                    [np.sin(theta), -1* np.cos(theta)]
                ])

    return UnitaryGate(Unitary_Matrix, label='U_gate({})'.format(np.around(theta, 3)))

In [ ]:
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate
N_QUBITS=3
q_reg = QuantumRegister(N_QUBITS)
qcirc = QuantumCircuit(q_reg)

U_gate = Arb_state_U_gate(np.pi/2).control(1)
U_gate.num_ctrl_qubits=2
U_gate.num_qubits=3
U_gate.ctrl_state=2
qcirc.append(U_gate, ([0,1,2]))
qcirc.draw()

In [ ]:
def Get_control_parameters_IBM(num_qubits, Coefficient_list):
    if len(Coefficient_list) != 2 ** num_qubits:
        # fill missing terms with amplitude of zero!
        Coefficient_list = Coefficient_list + [0 for _ in range(2 ** num_qubits - len(Coefficient_list))]
        #raise ValueError('incorrect number of coefficients')

    state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]

    alpha_j_dict = {}
    for target_qubit in range(num_qubits - 1):

        number_controls = target_qubit

        if number_controls > 0:
            CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2 ** number_controls)]
        else:
            CONTROL_state_list = ['']

        term_list = []
        for control_state in CONTROL_state_list:
            top_term_str = control_state + '1'
            bottom_term_str = control_state + '0'

            top = 0
            bottom = 0
            for index, state_str in enumerate(state_list):
                if state_str[:target_qubit + 1] == top_term_str:
                    top += Coefficient_list[index] ** 2

                if state_str[:target_qubit + 1] == bottom_term_str:
                    bottom += Coefficient_list[index] ** 2
                else:
                    continue

            if (bottom == 0) and (top == 0):
                angle = 0
            else:
                try:
                    angle = np.arctan(np.sqrt(top / bottom))
                except:
                    raise ValueError('undetermined angle! NEED TO CHECK PROBLEM')

            term_list.append({'control_state': control_state, 'angle': angle})
        alpha_j_dict[target_qubit] = term_list

    ##final rotation ##
    if num_qubits!=1:
        term_list = []
        for index, state_str in enumerate([Get_state_as_str((num_qubits - 1), i) for i in range(2 ** (num_qubits - 1))]):
            control_state_str = state_str

            top_term_str = control_state_str + '1'
            bottom_term_str = control_state_str + '0'

            index_top = state_list.index(top_term_str)
            index_bottom = state_list.index(bottom_term_str)

            top = Coefficient_list[index_top]
            bottom = Coefficient_list[index_bottom]

            if (bottom == 0) and (top == 0):
                angle = 0
            else:
                try:
                    angle = np.arctan(top / bottom)
                except:
                    raise ValueError('undetermined angle! NEED TO CHECK PROBLEM')

            term_list.append({'control_state': control_state_str, 'angle': angle})

        alpha_j_dict[num_qubits - 1] = term_list

        return alpha_j_dict
    else:

        # [np.cos(self.theta), np.sin(self.theta)],         [1]  =  [a]
        # [np.sin(self.theta), -1 * np.cos(self.theta)]     [0]     [b]
        theta = np.arccos(Coefficient_list[0])
        alpha_j_dict[0] = [{'control_state': '', 'angle': theta}]

        return alpha_j_dict

In [ ]:
test = Get_control_parameters_IBM(2,[np.sqrt(0.6), np.sqrt(0.1), np.sqrt(0.1), np.sqrt(0.2)])
test

In [ ]:
relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(2)[::-1])}
type(relabel_dic[0])

In [ ]:
def Get_state_prep_circuit_IBM2(circuit_param_dict, N_ancilla_qubits):
    
    q_reg = QuantumRegister(N_ancilla_qubits)
    qcirc = QuantumCircuit(q_reg)
    
    relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(N_ancilla_qubits)[::-1])}
    
    for qubit in circuit_param_dict:

        for term in circuit_param_dict[qubit]:
#             qubit = relabel_dic[qubit]
            if term['control_state']:
                control_value = int(term['control_state'],2)

                num_controls = len(term['control_state'])
                theta = term['angle']

                if theta == 0:
                    # yield cirq.I.on(cirq.LineQubit(qubit+self.N_system_qubits))
                    pass
                else:

                    U_gate = Arb_state_U_gate(theta).control(1)
                    U_gate.num_ctrl_qubits=num_controls
                    U_gate.num_qubits=num_controls+1
                    U_gate.ctrl_state= control_value
                    control_qubits = [relabel_dic[i] for i in range(U_gate.num_ctrl_qubits)]
                    active_qubit=relabel_dic[qubit]
                    qcirc.append(U_gate, ([*control_qubits, active_qubit]))
            else:
                theta = term['angle']
                if theta == 0:
                    continue # Identity 
                else:
                    active_qubit=relabel_dic[qubit]
                    U_gate = Arb_state_U_gate(theta)
                    qcirc.append(U_gate, ([active_qubit]))
                    
    return qcirc


In [ ]:
arb_state_circuit = Get_state_prep_circuit_IBM2(test, 2)
print(arb_state_circuit.draw())

In [ ]:
unitary= Operator(arb_state_circuit)
zero_input = reduce(np.kron, [np.array([[1],[0]]) for _ in range(arb_state_circuit.num_qubits)])
unitary.data.dot(zero_input)

In [ ]:
import random

In [ ]:
N_ancilla_qubits=3
amplitudes=np.array([random.uniform(0, 1) for _ in range(2**N_ancilla_qubits)])
amplitudes_normalized= amplitudes/np.sqrt(np.dot(amplitudes,amplitudes))

arb_control_dict = Get_control_parameters_IBM(N_ancilla_qubits,amplitudes_normalized)

arb_state_circuit = Get_state_prep_circuit_IBM2(arb_control_dict, N_ancilla_qubits)
print(arb_state_circuit.draw())

unitary= Operator(arb_state_circuit)
zero_input = reduce(np.kron, [np.array([[1],[0]]) for _ in range(arb_state_circuit.num_qubits)])
unitary.data.dot(zero_input)

In [ ]:
sum(np.abs(amplitudes_normalized)**2)

In [ ]:
amps = [np.sqrt(0.1), 
 np.sqrt(0.2),
 np.sqrt(0.3),
 np.sqrt(0.1),
 np.sqrt(0.05),
 np.sqrt(0.05),
 np.sqrt(0.1),
 np.sqrt(0.1)]

print(amps)

arb_control_dict = Get_control_parameters_IBM(N_ancilla_qubits,amps)

arb_state_circuit = Get_state_prep_circuit_IBM(arb_control_dict, N_ancilla_qubits)
print(arb_state_circuit.draw())

unitary= Operator(arb_state_circuit)
zero_input = reduce(np.kron, [np.array([[1],[0]]) for _ in range(arb_state_circuit.num_qubits)])
unitary.dot(zero_input)

In [ ]:
arb_control_dict

In [385]:
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate
N_QUBITS=2
q_reg = QuantumRegister(N_QUBITS+1)
qcirc = QuantumCircuit(q_reg)

X_gate = XGate().control(2)
X_gate.ctrl_state=0
qcirc.append(X_gate, [0,1,2])
qcirc.draw()

q3021_0: ──o──
           │  
q3021_1: ──o──
         ┌─┴─┐
q3021_2: ┤ X ├
         └───┘